Prepare for y4q1 (Oct-Dec submission)

First issue. Starting to register EN-TEC biosamples. which have a fairly complex relationships

In [1]:
import pandas
import gcat
import paramiko

In [2]:
from curation_common import *

In [3]:
server = ENCODED('www.encodeproject.org')
server.load_netrc()

In [4]:
spreadsheet_name = 'ENCODE Submission for Y4Q1'

# Register Biosamples

In [5]:
#y4q1 = gcat.get_file(spreadsheet_name, fmt='pandas_excel')
#biosample = y4q1.parse('Biosamples', header=0)
#created = server.post_sheet('/biosamples/', biosample, verbose=True, dry_run=True)
#print(len(created))

In [6]:
#biosample.to_excel('/dev/shm/biosamples.xlsx', index=False)

# Register Experiments

In [7]:
opts = gcat.default_options()

In [8]:
opts['title'] = spreadsheet_name

In [9]:
opts.update(gcat.load_config(opts))

In [10]:
service = gcat.get_service(opts)

In [11]:
files = service.files()
req = files.list()

In [12]:
res = req.execute()

In [13]:
[ x.get('title') for x in res['items']]

['HudsonAlpha END GAME 2016',
 'ZPF1_iso1-FLAG_HepG2 PCR validation.jpg',
 'ENCODE Production Plans',
 'FLAG-PREB_HepG2',
 'PREB-HepG2.xlsx',
 'PREB-FLAG_HepG2 PCR validation Rep2 only.jpg',
 'PREB-FLAG_HepG2 caption Rep2 only.docx',
 'FLAG-TAG_sorted.csv',
 'FLAG-PREB_HepG2_Mass Spec Caption.docx',
 'FLAG-PREB_HepG2_IP-MS.pdf',
 'FLAG-NCoA2_HepG2',
 'NCoA2-HepG2.xlsx',
 'NCoA2-FLAG_HepG2 PCR validation.jpg',
 'NCoA2-FLAG_HepG2 caption.docx',
 'FLAG-TAG_sorted.csv',
 'FLAG-NCoA2_HepG2_Mass Spec Caption.docx',
 'FLAG-NCoA2_HepG2_IP-MS.pdf',
 'FLAG-TEAD1_HepG2',
 'TEAD1-HepG2.xlsx',
 'TEAD1-FLAG_HepG2 PCR validation.jpg',
 'TEAD1-FLAG_HepG2 caption.docx',
 'FLAG-TEAD1_HepG2_Mass Spec Caption.docx',
 'FLAG-TEAD1_HepG2_IP-MS.pdf',
 'FLAG-TAG_sorted.csv',
 'FLAG-CEBPG_HepG2',
 'FLAG-TAG_sorted.csv',
 'FLAG-CEBPG_HepG2_Mass Spec Caption.docx',
 'FLAG-CEBPG_HepG2_IP-MS.pdf',
 'CEBPG-HepG2.xlsx',
 'CEBPG-FLAG_HepG2 PCR validation.jpg',
 'CEBPG-FLAG_HepG2 caption.docx',
 'FLAG-MIXL1_HepG2',
 'M

In [14]:
res = files.list_next(req, res).execute()

In [15]:
len([ x.get('title') for x in res['items']])

100

In [16]:
res = service.files()

In [17]:
y4q1 = gcat.get_file(spreadsheet_name, fmt='pandas_excel')
experiments = y4q1.parse('Experiments', header=0)

created = server.post_sheet('/experiments/', experiments, verbose=True, dry_run=True)

In [18]:
len(created)

0

In [19]:
if created:
    experiments.to_excel('/tmp/experiments.xlsx', index=False)

# Register Libraries

In [20]:
y4q1 = gcat.get_file(spreadsheet_name, fmt='pandas_excel')
libraries = y4q1.parse('Libraries', header=0)

created = server.post_sheet('/libraries/', libraries, verbose=True, dry_run=True)

In [21]:
len(created)

0

In [22]:
if created:
    libraries.to_excel('/tmp/libraries.xlsx', index=False)

# Register Replicates

In [23]:
y4q1 = gcat.get_file(spreadsheet_name, fmt='pandas_excel')
replicates = y4q1.parse('Replicates', header=0)

created = server.post_sheet('/replicates/', replicates, verbose=True, dry_run=True)

In [24]:
len(created)

0

In [25]:
if created:
    replicates.to_excel('/tmp/replicates.xlsx', index=False)

# Create Manifest

In [26]:
def make_manifest(spreadsheet_name, submission_name, submission_root, map_name):
    book = gcat.get_file(spreadsheet_name, fmt='pandas_excel')
    libraries = book.parse('Libraries')
    replicates = book.parse('Replicates')
    
    pongo = paramiko.SSHClient()
    pongo.set_missing_host_key_policy(paramiko.WarningPolicy())
    pongo.connect('pongo.cacr.caltech.edu', username='diane')
    sftp = pongo.open_sftp()
    sftp.chdir(submission_root)
    files = 0
    for line in sftp.open(map_name):
        # skip comments
        line = line.strip()
        if len(line) == 0 or line.startswith('#'):
            continue

        library_id, dirname = line.split()
        alias = 'barbara-wold:{}'.format(library_id)
        library_row = libraries[libraries['aliases:array'] == alias]
        replicate_row = replicates[replicates['library'] == alias]
        for filename in sftp.listdir(dirname):
            if filename.endswith('fastq.gz'):
                relative_path = os.path.join(dirname, filename)
                metadata = {
                    'dataset': list(replicate_row['experiment'])[0],
                    'submitted_file_name': relative_path,
                    'replicate': list(replicate_row['uuid'])[0],
                    'lab': list(library_row['lab'])[0],
                    'award': list(library_row['award'])[0],
                }
                #display.display_pretty(metadata)
                print('<http://jumpgate.caltech.edu/wiki/SubmissionsLog/{}#{}>'.format(submission_name, dirname))
                print('  encode3:dataset "{}" ;'.format(metadata['dataset']))
                print('  encode3:replicate "{}" ;'.format(metadata['replicate']))
                print('  encode3:lab "{}" ;'.format(metadata['lab']))
                print('  encode3:award "{}" .'.format(metadata['award']))

                print('<file:///woldlab/castor/home/diane/proj/submission/{}/{}>'.format(submission_name, relative_path))
                print('  encode3:output_type "reads" ;')
                print('  encode3:file_format "fastq" .')
                files += 1

    print('generated {} records'.format(files))


In [27]:
make_manifest(spreadsheet_name, 'encode-201604', '/home/diane/proj/submission/encode-201604/', 'libs.txt')

/usr/lib/python3/dist-packages/paramiko/client.py:658: UserWarning: Unknown ssh-rsa host key for pongo.cacr.caltech.edu: b'6833531597d6176b45c27f33e9406a12'
  (key.get_name(), hostname, hexlify(key.get_fingerprint())))


<http://jumpgate.caltech.edu/wiki/SubmissionsLog/encode-201604#16922>
  encode3:dataset "ENCSR647QBV" ;
  encode3:replicate "3d27f8a5-bc57-4864-9220-35c81e4ec94d" ;
  encode3:lab "barbara-wold" ;
  encode3:award "U54HG006998" .
<file:///woldlab/castor/home/diane/proj/submission/encode-201604/16922/16922_HLHGNBCXX_c101_l1.fastq.gz>
  encode3:output_type "reads" ;
  encode3:file_format "fastq" .
<http://jumpgate.caltech.edu/wiki/SubmissionsLog/encode-201604#16922>
  encode3:dataset "ENCSR647QBV" ;
  encode3:replicate "3d27f8a5-bc57-4864-9220-35c81e4ec94d" ;
  encode3:lab "barbara-wold" ;
  encode3:award "U54HG006998" .
<file:///woldlab/castor/home/diane/proj/submission/encode-201604/16922/16922_HLHGNBCXX_c116_l2.fastq.gz>
  encode3:output_type "reads" ;
  encode3:file_format "fastq" .
<http://jumpgate.caltech.edu/wiki/SubmissionsLog/encode-201604#16923>
  encode3:dataset "ENCSR647QBV" ;
  encode3:replicate "891a2a57-8add-4e1a-ac0b-fc0c45c0c5ec" ;
  encode3:lab "barbara-wold" ;
  encode3: